In [2]:
import os
import pandas as pd

In [3]:
# load cases, links and legislation
inputpath = '/media/sf_VBox_Shared/CaseLaw/graphs/lido/'
cases = pd.read_csv(os.path.join(inputpath, 'hr_simple_nodes.csv'))
legislation = pd.read_csv(os.path.join(inputpath, 'hr_simple_legislation_links.csv'))
cases_links = pd.read_csv(os.path.join(inputpath, 'hr_simple_links.csv'))

In [4]:
leg_per_node = legislation.groupby('source')['title'].apply(lambda l: "|".join(list(sorted(l))))

In [5]:
cases = cases.set_index('id')
cases['leg'] = leg_per_node
cases = cases.reset_index()

In [6]:
legislation.groupby('title')['source'].nunique().sort_values(ascending=False).head(20)

title
Wet op de rechterlijke organisatie, Artikel 81     12732
Wetboek van Strafvordering                          2043
Wetboek van Strafrecht                              1395
Wet op de rechterlijke organisatie, Artikel 80a     1222
Wet waardering onroerende zaken                      708
Opiumwet                                             624
Wetboek van Strafvordering, Artikel 457              614
Wetboek van Strafvordering, Artikel 359              603
Wetboek van Strafvordering, Artikel 552a             567
Wetboek van Strafvordering, Artikel 440              432
Wet op de omzetbelasting 1968                        413
Burgerlijk Wetboek Boek 6                            410
Wetboek van Burgerlijke Rechtsvordering              402
Algemene wet inzake rijksbelastingen                 400
Wetboek van Strafvordering, Artikel 365a             397
Wetboek van Strafvordering, Artikel 359a             338
Algemene wet bestuursrecht, Artikel 8:41             337
Algemene wet bestuursrech

## Investigate legislation network

In [10]:
legislation.columns

Index(['link_id', 'source', 'article', 'link_type', 'link_type_label',
       'title'],
      dtype='object')

In [11]:
legislation.groupby('title')['article'].nunique().sort_values(ascending=False)

title
Wetboek van Strafvordering                                                                  81
Wetboek van Strafrecht                                                                      70
Algemene wet bestuursrecht                                                                  68
Wet inkomstenbelasting 2001                                                                 54
Algemene wet inzake rijksbelastingen                                                        50
Wet milieubeheer                                                                            49
Wegenverkeerswet 1994                                                                       46
Gemeentewet                                                                                 45
Wet op de vennootschapsbelasting 1969                                                       43
Wetboek van Burgerlijke Rechtsvordering                                                     41
Invorderingswet 1990                        

In [12]:
legislation_title_references = legislation[['source', 'title']].drop_duplicates()

In [13]:
legislation_title_references.shape

(71834, 2)

In [14]:
legislation_title_references.to_csv(os.path.join(inputpath, 'hr_case_leg_titles.csv'))

## Investigate hierarcy

In [60]:
legislation_title_references.head()

,source,title
0,http://linkeddata.overheid.nl/terms/jurisprude...,"Wet inkomstenbelasting 2001, Artikel 2.3"
1,http://linkeddata.overheid.nl/terms/jurisprude...,"Wet inkomstenbelasting 2001, Artikel 2.7"
2,http://linkeddata.overheid.nl/terms/jurisprude...,"Wet inkomstenbelasting 2001, Artikel 6.2"
3,http://linkeddata.overheid.nl/terms/jurisprude...,"Wet inkomstenbelasting 2001, Artikel 6.24"
4,http://linkeddata.overheid.nl/terms/jurisprude...,"Wet inkomstenbelasting 2001, Artikel 6.39"


In [66]:
high_title = 'Wetboek van Strafrecht'
referring_sources = legislation_title_references[legislation_title_references['title']==high_title]['source'].unique()
legislation_title_references[legislation_title_references['source'].isin(referring_sources)].groupby('source').nunique('title')

,source,title
source,,
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:1958:AG2051,1,2
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:1997:AG3125,1,8
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:1998:ZD9060,1,2
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:2001:AB0576,1,4
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:2001:AB2961,1,8
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:2001:AF9978,1,6
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:2001:AF9985,1,2
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:2001:AL6496,1,2
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:HR:2001:AL6511,1,2


# Collapse network

In [67]:
import scipy.sparse

In [68]:
ltr_df = legislation_title_references.set_index(['source', 'title'])
ltr_df['cnt'] = 1
case_leg_adj_df = ltr_df.unstack(fill_value=0)

In [69]:
case_leg_adj_m = scipy.sparse.coo_matrix(case_leg_adj_df.values)

In [70]:
case_leg_adj_m.shape

(25446, 7171)

In [71]:
case_to_case = case_leg_adj_m.dot(case_leg_adj_m.T)
leg_to_leg = case_leg_adj_m.T.dot(case_leg_adj_m)

In [72]:
case_to_case = case_to_case.asformat('coo')

In [34]:
leg_to_leg = leg_to_leg.tocoo()

In [77]:
leg_to_leg = scipy.sparse.triu(leg_to_leg, 1)

case_to_case = scipy.sparse.triu(case_to_case, 1)

In [79]:
case_to_case, leg_to_leg

(<25446x25446 sparse matrix of type '<class 'numpy.int64'>'
 	with 86460934 stored elements in COOrdinate format>,
 <7171x7171 sparse matrix of type '<class 'numpy.int64'>'
 	with 70999 stored elements in COOrdinate format>)

In [91]:
leg_labels = pd.Series(case_leg_adj_df.columns.get_level_values(1)).as_matrix()

In [80]:

sources = leg_labels[leg_to_leg.row]
targets = leg_labels[leg_to_leg.col]
values = leg_to_leg.data

In [81]:
len(sources), len(targets), len(values)

(70999, 70999, 70999)

In [82]:
leg_to_leg_df = pd.DataFrame(data={'source': sources, 'target': targets, 'cnt': values}) # columns=['source', 'target', 'cnt'])
leg_to_leg_df.head()

,cnt,source,target
0,1,Aanbestedingswet 2012,"Aanbestedingswet 2012, Artikel 4.30"
1,1,Aanbestedingswet 2012,"Aanbestedingswet 2012, Artikel 1.4"
2,1,Aanbestedingswet 2012,Wet implementatie rechtsbeschermingsrichtlijne...
3,1,Aanbestedingswet 2012,Wet implementatie rechtsbeschermingsrichtlijne...
4,1,Aanbestedingswet 2012,Wet implementatie rechtsbeschermingsrichtlijne...


In [84]:
leg_to_leg_df.to_csv(os.path.join(inputpath, '2018-01-03 leg_to_leg_titles.csv'), index=False)

In [85]:
leg_to_leg_df_min10 = leg_to_leg_df[leg_to_leg_df['cnt']>=10]
leg_to_leg_df_min10.to_csv(os.path.join(inputpath, '2018-01-03 leg_to_leg_titles_min10.tsv'), index=False, sep='\t')

In [ ]:
case_to_case_min2 

In [86]:
case_labels = pd.Series(case_leg_adj_df.index).as_matrix()
sources = case_labels[case_to_case.row]
targets = case_labels[case_to_case.col]
values = case_to_case.data

In [88]:
# TODO: this is a very dense network
case_to_case_df = pd.DataFrame([sources, targets, values], columns=['source', 'target', 'cnt'])
case_to_case_df.head()

MemoryError: 

## Itemsets

In [74]:
from orangecontrib.associate import fpgrowth

In [102]:
itemsets = fpgrowth.frequent_itemsets(case_leg_adj_m, min_support=10)
rules = fpgrowth.association_rules(dict(itemsets), min_confidence=0.9)

In [103]:
for ante, conse, support, conf in list(rules):
    if(len(ante)<=1):
        print(leg_labels[list(ante)], '->', leg_labels[list(conse)], support, conf)

['Wet voorkeursrecht gemeenten, Artikel 14'] -> ['Wet voorkeursrecht gemeenten' 'Wet voorkeursrecht gemeenten, Artikel 26'
 'Wet voorkeursrecht gemeenten, Artikel 27'
 'Wet voorkeursrecht gemeenten, Artikel 10'
 'Wet op de rechterlijke organisatie, Artikel 81'] 10 1.0
['Wet voorkeursrecht gemeenten, Artikel 14'] -> ['Wet voorkeursrecht gemeenten' 'Wet voorkeursrecht gemeenten, Artikel 26'
 'Wet voorkeursrecht gemeenten, Artikel 27'
 'Wet op de rechterlijke organisatie, Artikel 81'] 10 1.0
['Wet voorkeursrecht gemeenten, Artikel 14'] -> ['Wet voorkeursrecht gemeenten' 'Wet voorkeursrecht gemeenten, Artikel 26'
 'Wet voorkeursrecht gemeenten, Artikel 27'
 'Wet voorkeursrecht gemeenten, Artikel 10'] 10 1.0
['Wet voorkeursrecht gemeenten, Artikel 14'] -> ['Wet op de rechterlijke organisatie, Artikel 81'
 'Wet voorkeursrecht gemeenten, Artikel 26'
 'Wet voorkeursrecht gemeenten, Artikel 27'
 'Wet voorkeursrecht gemeenten, Artikel 10'] 10 1.0
['Wet voorkeursrecht gemeenten, Artikel 14'] -> [

## Create subnetwork

In [22]:
def create_subnetwork_and(articles, legislation, nodes_in, links_in):
    case_ids = legislation[legislation['title'].isin(articles)]['source'].unique()
    nodes_sub = nodes_in[nodes_in['id'].isin(case_ids)]
    links_sub = links_in[links_in['source'].isin(case_ids)]
    links_sub = links_sub[links_sub['target'].isin(case_ids)]
    return nodes_sub, links_sub

In [57]:
def create_subnetwork_or(articles, legislation, nodes_in, links_in):
    case_ids = legislation[legislation['title'].isin(articles)]['source'].unique()
    links_sub = links_in[links_in['source'].isin(case_ids) | links_in['target'].isin(case_ids)]
    case_ids_extended = set(links_sub['source']).union(set(links_sub['target'])).union(case_ids)
    nodes_sub = nodes_in[nodes_in['id'].isin(case_ids_extended)].copy()
    nodes_sub['in_set'] = nodes_sub['id'].isin(case_ids)
    return nodes_sub, links_sub

In [58]:
nodes_wga, links_wga = create_subnetwork_and(
    ["Burgerlijk Wetboek Boek 7, Artikel 658", "Burgerlijk Wetboek Boek 7, Artikel 611"], 
    legislation, cases, cases_links)
print(nodes_wga.shape, links_wga.shape)

(127, 6) (53, 5)


In [59]:
nodes_wga_extended, links_wga_extended = create_subnetwork_or(
    ["Burgerlijk Wetboek Boek 7, Artikel 658", "Burgerlijk Wetboek Boek 7, Artikel 611"], 
    legislation, cases, cases_links)
print(nodes_wga_extended.shape, links_wga_extended.shape)

(197, 7) (154, 5)


In [9]:
nodes_81ro, links_81ro = create_subnetwork(
    ["Wet op de rechterlijke organisatie, Artikel 81"],
    legislation, cases, cases_links)
print(nodes_81ro.shape, links_81ro.shape)

(12732, 6) (1380, 5)
